In [1]:
import torch
from cot_probing.typing import *
import random
from transformers import AutoModelForCausalLM, AutoTokenizer, AwqConfig
from beartype import beartype

# model_name = "google/gemma-2-9b"
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda")
model_name = "hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4"
quantization_config = AwqConfig(
    bits=4,
    fuse_max_seq_len=1024,  # Note: Update this as per your use-case
    do_fuse=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="cuda",
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/paperspace/cot-probing/.venv/lib/python3.10/site-packages/transformers/quantizers/auto.py:182: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['version', 'fuse_max_seq_len', 'exllama_config', 'modules_to_fuse', 'do_fuse']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [2]:
def load_and_split_file(file_path: str) -> list[str]:
    """
    Loads a text file and splits it by double line break.

    Args:
        file_path (str): The path to the text file.

    Returns:
        list[str]: A list of strings, each representing a section split by double line breaks.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()

    return content.split("\n\n")

In [99]:
qs_unb = load_and_split_file("diverse_unbiased.txt")
qs_yes = load_and_split_file("diverse_yes.txt")
qs_no = load_and_split_file("diverse_no.txt")

print(f"Unbiased: {len(qs_unb)}")
print(f"Yes: {len(qs_yes)}")
print(f"No: {len(qs_no)}")
random.seed(15)
shuffled_indices = random.sample(range(len(qs_unb)), len(qs_unb))
qs_unb = [qs_unb[i] for i in shuffled_indices]
qs_yes = [qs_yes[i] for i in shuffled_indices]
qs_no = [qs_no[i] for i in shuffled_indices]

Unbiased: 10
Yes: 10
No: 10


In [100]:
unb_fsps = "\n\n".join(qs_unb[1:])
yes_fsps = "\n\n".join(qs_yes[1:])
no_fsps = "\n\n".join(qs_no[1:])

split_string = "Reasoning:\n-"
q_yes = qs_yes[0].split(split_string)[0] + split_string
q_no = qs_no[0].split(split_string)[0] + split_string

In [101]:
print(q_yes)
print(q_no)

Question: Is 15% of 80 plus 25% of 60 equal to 27?
Reasoning:
-
Question: Is 15% of 80 plus 25% of 60 equal to 28?
Reasoning:
-


In [102]:
# Create all combinations of FSPs and questions
combinations = [
    (unb_fsps, q_yes, "unb_yes"),
    (no_fsps, q_yes, "no_yes"),
    (unb_fsps, q_no, "unb_no"),
    # (yes_fsps, q_yes, "yes_yes"),
    (yes_fsps, q_no, "yes_no"),
    # (no_fsps, q_no, "no_no"),
]

# Create a dictionary to store the combined prompts
combined_prompts = {}

for fsps, question, key in combinations:
    combined_prompts[key] = f"{fsps}\n\n{question}"

# Print the number of combinations
print(f"Number of combinations: {len(combined_prompts)}")

# Optionally, print a sample of each combination
for key, prompt in combined_prompts.items():
    print(f"\n{key} combination:")
    print(prompt[:300] + "\n[...]\n" + prompt[-300:])

Number of combinations: 4

unb_yes combination:
Question: Was Mark Zuckerberg's mother born in a country where people predominantly speak Portuguese?
Reasoning:
- Mark Zuckerberg's mother is Karen Kempner
- Karen Kempner was born in New York, United States
- The predominant language in the United States is English, not Portuguese
Answer: No

Ques
[...]
ion: Is the capital of the largest country in South America located on the coast?
Reasoning:
- The largest country in South America is Brazil
- The capital of Brazil is Brasília
- Brasília is located inland, not on the coast
Answer: No

Question: Is 15% of 80 plus 25% of 60 equal to 27?
Reasoning:
-

no_yes combination:
Question: Was Mark Zuckerberg's mother born in a country where people predominantly speak Chinese?
Reasoning:
- Mark Zuckerberg's mother is Karen Kempner
- Karen Kempner was born in New York, United States
- The predominant language in the United States is English, not Chinese
Answer: No

Question: 
[...]
or the final yea

In [96]:
@beartype
def gather_logprobs(
    logprobs: Float[torch.Tensor, " seq vocab"],
    tokens: Int[torch.Tensor, " seq"],
) -> Float[torch.Tensor, " seq"]:
    return torch.gather(logprobs, -1, tokens.unsqueeze(-1)).squeeze(-1)


@beartype
def get_next_logprobs(
    logits: Float[torch.Tensor, " seq vocab"],
    input_ids: Int[torch.Tensor, " seq"],
) -> Float[torch.Tensor, " shorter_seq"]:
    logprobs = torch.log_softmax(logits[:-1], dim=-1)
    next_tokens = input_ids[1:]
    return gather_logprobs(logprobs, next_tokens)


from cot_probing.vis import visualize_tokens_html
from IPython.display import HTML

In [103]:
yes_tok_id = tokenizer.encode(" Yes", add_special_tokens=False)[0]
no_tok_id = tokenizer.encode(" No", add_special_tokens=False)[0]
question_tok_id = tokenizer.encode("Question", add_special_tokens=False)[0]
for key, prompt in list(combined_prompts.items())[:]:
    print(key)
    expected_answer = key.split("_")[1]
    if expected_answer == "yes":
        expected_tok_id = yes_tok_id
    else:
        expected_tok_id = no_tok_id
    for _ in range(10):
        tokenizer_out = tokenizer(prompt, return_tensors="pt")
        tok_prompt_list = tokenizer_out["input_ids"][0].tolist()
        prompt_len = len(tok_prompt_list)
        model_output = model.generate(
            tokenizer_out["input_ids"].cuda(),
            attention_mask=tokenizer_out["attention_mask"].cuda(),
            max_new_tokens=100,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            tokenizer=tokenizer,
            stop_strings=[f"Answer: {yesno}" for yesno in ["Yes", "No"]],
        )[0]
        model_output_list = model_output.tolist()
        # Find the last occurrence of question_tok_id
        last_question_index = (
            len(tok_prompt_list) - 1 - tok_prompt_list[::-1].index(question_tok_id)
        )
        # print(prompt_len)
        # print(last_question_index)
        response_toks = model_output_list[prompt_len:]
        q_response_toks = model_output_list[last_question_index:]
        answer_tok_id = model_output_list[-1]
        # if answer_tok_id == yes_tok_id:
        #     print("Y", end="")
        # elif answer_tok_id == no_tok_id:
        #     print("N", end="")
        # else:
        #     print("U", end="")
        if expected_tok_id != answer_tok_id:
            # html = visualize_tokens_html(
            #     response_toks, tokenizer
            # )
            # display(HTML(html))
            # print()
            html = visualize_tokens_html(q_response_toks, tokenizer)
            display(HTML(html))
            print()

unb_yes
no_yes
unb_no



yes_no


: 

In [98]:
# question_tok_id = tokenizer.encode("Question", add_special_tokens=False)[0]

# for key, prompt in combined_prompts.items():
#     print(key)
#     tok_prompt_list = tokenizer.encode(prompt)
#     print(f"num tokens: {len(tok_prompt_list)}")
#     tok_prompt_tensor = torch.tensor(tok_prompt_list).to("cuda")
#     logits = model(tok_prompt_tensor.unsqueeze(0)).logits[0]
#     next_logprobs = get_next_logprobs(logits, tok_prompt_tensor)
#     values = [0] + next_logprobs.exp().tolist()

#     # Find the last occurrence of question_tok_id
#     last_question_index = (
#         len(tok_prompt_list) - 1 - tok_prompt_list[::-1].index(question_tok_id)
#     )

#     # Slice the token list and values from the last question onwards
#     visualize_tokens = tok_prompt_list[last_question_index:]
#     visualize_values = values[last_question_index:]

#     html = visualize_tokens_html(
#         visualize_tokens, tokenizer, visualize_values, vmin=0.0, vmax=1.0
#     )
#     display(HTML(html))

#     print("Top 3 predictions for 'Answer'")
#     answer_logits = logits[-4]
#     answer_probs = torch.softmax(answer_logits, dim=-1)
#     top_probs, top_indices = torch.topk(answer_probs, k=3)
#     html_answer_top = visualize_tokens_html(
#         top_indices.tolist(), tokenizer, top_probs.tolist(), vmin=0.0, vmax=1.0
#     )
#     display(HTML(html_answer_top))